#Preparation

#####Importing libraries and mounting Google Drive

In [1]:
import json
import os, os.path
import csv
import requests
from datetime import timedelta, date, datetime
from bs4 import BeautifulSoup
import urllib.request
from urllib.request import urlopen
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


#####Defining function

In [2]:
#function to automatically generate the date range from the parameter
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)
        
#function to help converting python dictionary to json file
def python_dict_to_json_file(file_path):
    try:
        # Get a file object with write permission.
        file_object = open(file_path, 'w')
        #dict_object = dict(name='Tom', age=25, score=100)
        # Save dict data into the JSON file.
        json.dump(raw_data, file_object)
        #print(file_path + ' created. ')
    except FileNotFoundError:
        print(file_path + ' not found. ')

#####Setting parameter for crawling

In [3]:
root = '/content/drive/My Drive/Colab Notebooks/FlickrCrawling/Sarbagita'

#defining parameter for the API
api_key = '_________________________'
year = '2020'
minimum_taken_date = date(2019, 12, 1)
maximum_taken_date = date(2020, 1, 1)
lower_longitude = '114.416433'
lower_latitude = '-8.875014'
upper_longitude = '115.725141'
upper_latitude = '-8.045447'
bbox = lower_longitude+','+lower_latitude+','+upper_longitude+','+upper_latitude
# bbox_south_west_point = '115.05,8.866667'
# bbox_north_east_point = '115.3833,-8.433333'
accuracy = '12' #World level is 1, Country is ~3, Region is ~6, City is ~11, Street is ~16
has_geo = '1' #1 for geotagged photo, 0 for photo without geotag
step = 2 #range of days between each data crawling attempt

#defining date range for data
date_range = []
for single_date in daterange(minimum_taken_date, maximum_taken_date):
    single_date_string = single_date.strftime('%Y-%m-%d')
    date_range.append(single_date_string)
date_length = len(date_range)
print(date_length)
print('date range: ' + date_range[0] + ' - ' + date_range[-1])

31
date range: 2019-12-01 - 2019-12-31


#Data Crawling

#####Using 'photos.search' function from to collect data matching the defined parameter and exporting the result into JSON files

In [4]:
url = 'https://api.flickr.com/services/rest/?method=flickr.photos.search'

i = 0
j = step - 1

while True:
    try:
        while j < date_length:
            params = dict(api_key=api_key,
                          format='json',
                          nojsoncallback='1',
                          min_taken_date=date_range[i],
                          max_taken_date=date_range[j],
                          bbox=bbox,
                          accuracy=accuracy,
                          has_geo=has_geo)
            resp = requests.get(url=url, params=params)
            data = json.loads(resp.text)

            file_object = open(root+'/JSON/photos.search/{0}/{1} - {2}.json' .format(year, date_range[i], date_range[j]), 'w')
            json.dump(data, file_object)

            print('{0} - {1}.json created.' .format(date_range[i], date_range[j]))
            # print(json.dumps(data, sort_keys=True, indent=4))
            i += step
            j += step
    except Exception as ex:
        print('{0} when trying to get data of {0} - {1}. Retrying...' .format(ex, date_range[i], date_range[j]))
        continue
    break

2019-12-01 - 2019-12-02.json created.
2019-12-03 - 2019-12-04.json created.
2019-12-05 - 2019-12-06.json created.
2019-12-07 - 2019-12-08.json created.
2019-12-09 - 2019-12-10.json created.
2019-12-11 - 2019-12-12.json created.
2019-12-13 - 2019-12-14.json created.
2019-12-15 - 2019-12-16.json created.
2019-12-17 - 2019-12-18.json created.
2019-12-19 - 2019-12-20.json created.
2019-12-21 - 2019-12-22.json created.
2019-12-23 - 2019-12-24.json created.
2019-12-25 - 2019-12-26.json created.
2019-12-27 - 2019-12-28.json created.
2019-12-29 - 2019-12-30.json created.


#####Taking the 'photo_id', 'owner', and 'secret' from the previous JSON files and combining it into a python dictionaries list

In [5]:
DIR = root+'/JSON/photos.search/{0}' .format(year)
photos_search_count = len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))])
x = 0
i = 0
j = step - 1
dataset = []

while True:
    try:
        while x < photos_search_count:
            with open(root+'/JSON/photos.search/{0}/{1} - {2}.json' .format(year, date_range[i], date_range[j])) as data_file:
                n = 0
                jdata = json.load(data_file)
                jdata_len = len(jdata['photos']['photo'])
                while n < jdata_len:
                    new_corpus = {'id':jdata['photos']['photo'][n]['id'],
                                    'owner':jdata['photos']['photo'][n]['owner'],
                                    'secret':jdata['photos']['photo'][n]['secret']}
                    dataset.append(new_corpus)
                    n += 1
                #print('{0} corpus added from: {1} - {2}.json added.' .format(jdata_len, date_range[i], date_range[j]))
                x += 1
                i += step
                j += step
                n = 0
    except Exception as ex:
        print('{0} when trying to get data from to {1}. Retrying...' .format(ex, data_file))
    break

print('Total corpus added:', len(dataset))

Expecting value: line 1 column 1 (char 0) when trying to get data from to <_io.TextIOWrapper name='/content/drive/My Drive/Colab Notebooks/FlickrCrawling/Sarbagita/JSON/photos.search/2020/2019-12-29 - 2019-12-30.json' mode='r' encoding='UTF-8'>. Retrying...
Total corpus added: 449


#####Getting detailed data of each photo using the 'getInfo' function and data from the previous dictioary and exporting it to JSON files

In [6]:
url = 'https://api.flickr.com/services/rest/?method=flickr.photos.getInfo'
i = 0
dataset_length = len(dataset)
error_count = 0

while True:
    try:
        while i < dataset_length:
            params = dict(api_key=api_key,
                          format='json',
                          nojsoncallback='1',
                          photo_id=dataset[i]['id'],
                          secret=dataset[i]['secret'])
            resp = requests.get(url=url, params=params)
            data = json.loads(resp.text)
            # print(json.dumps(data, sort_keys=True, indent=4))

            file_object = open(root+'/JSON/getInfo/{0}/{1}.json' .format(year, dataset[i]['id']), 'w')
            json.dump(data, file_object)
            
            i += 1
            if i % 200 == 0:
                print('{0} / {1} JSON data created.' .format(i, dataset_length))
            error_count = 0
    except Exception as ex:
        error_count += 1
        print('{0} when trying to get data of {1}. Retrying...' .format(ex, dataset[i]['id']))
        if error_count > 5: pass
        else: continue
    break

print('Total JSON data created: {0}' .format(i))  

200 / 449 JSON data created.
400 / 449 JSON data created.
Total JSON data created: 449


#####Taking the 'photo_id', 'owner_NSID', 'owner_location', 'date_taken', and 'location' from the previous json files and combining it into a single JSON file

Note: Use [CodeBeautify](https://codebeautify.org/jsonviewer) to clean the JSON file and export it to csv format to make it easier to read and cleanse.

In [7]:
i = 0
DIR = root+'/JSON/getInfo/{0}' .format(year)
file_list = [f for f in os.listdir(DIR) if '.json' in f]
num_files = len(file_list)
csv_columns = ['photo_id', 'owner_nsid', 'owner_location', 'dates_taken', 'location_latitude', 'location_longitude', 'description']
getinfo = []
print('Files found in the directory: ', num_files)

while True:
    try:
        # while i < num_files:
        for f in file_list:
            # with open(root+'/JSON/getInfo/{0}/{1}.json' .format(year, dataset[i]['id'])) as data_file:
            with open(root+'/JSON/getInfo/{0}/{1}' .format(year, f)) as data_file:
                jdata = json.load(data_file)
                new_corpus = {'photo_id':jdata['photo']['id'],
                              'owner_nsid':jdata['photo']['owner']['nsid'],
                              'owner_location':jdata['photo']['owner']['location'],
                              'dates_taken':jdata['photo']['dates']['taken'],
                              'location_latitude':jdata['photo']['location']['latitude'],
                              'location_longitude':jdata['photo']['location']['longitude'],
                              'description':jdata['photo']['description']['_content']}
                getinfo.append(new_corpus)
                #print('Corpus from: {0}.json is added' .format(dataset[i]['id']))
                i += 1
                if i % 500 == 0:
                    print('{0} / {1} files added to dictionary.' .format(i, num_files))
                    continue
    except Exception:
        print('{0} when trying to get data from to {1}' .format(Exception, f))
        # print('{0} when trying to get data from to {1}.json' .format(Exception, dataset[i]['id']))
        pass
    break

# print('{0} / {1} files added to dictionary.' .format(i, num_files))

try:
    with open(root+'/CSV/dirty/{0}_dirty.csv' .format(year), 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in getinfo:
            writer.writerow(data)
except IOError:
    print("I/O error")

print('A new CSV file containing {0} rows has been created.' .format(i))

Files found in the directory:  514
<class 'Exception'> when trying to get data from to 49285198212.json
A new CSV file containing 456 rows has been created.


#Download image

In [ ]:
df = pd.read_csv(root+'/CSV/2019-2020_ready.csv')
df['description'] = ''

for i, row in df.iterrows():
    try:
        with open(root+'/JSON/getInfo/2019/{0}.json' .format(row['photo_id'])) as data_file: jdata = json.load(data_file)
    except FileNotFoundError:
        with open(root+'/JSON/getInfo/2020/{0}.json' .format(row['photo_id'])) as data_file: jdata = json.load(data_file)
    df.at[i, 'description'] = jdata['photo']['description']['_content']
    print(row['photo_id'], jdata['photo']['description']['_content'])

# df.to_csv(root+'/CSV/2019-2020_ready(1).csv')
# df.head()

photo_id = df['photo_id']

for idx in photo_id:
    with open(root+'/JSON/getInfo/{0}/{1}.json' .format(year, idx)) as data_file:    
        jdata = json.load(data_file)
    url = jdata['photo']['urls']['url'][0]['_content']
        
    html = urlopen(url)
    soup = BeautifulSoup(html)

    img = soup.find('img', {'class':'low-res-photo'})
    urllib.request.urlretrieve('https:'+img['src'], root+'/IMG/{0}/{1}_{2}.jpg' .format(year, jdata['photo']['dates']['taken'][:10], idx))

    if idx % 100 == 0: print('{0} / {1} images downloaded.' .format(i, len(photo_id)))